In [1]:
import os
import pandas as pd
from src.processer_file import read_path 
from src.processer_file import get_path 
from src.processer_file import export_file

export_file.export_to_csv(98, 'igv')

('ICA', 'ica20240402', 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W98\\ica20240402\\ica01Data.csv')
('ICA', 'ica20240402', 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W98\\ica20240402\\ica03Data.csv')
('ICA', 'ica20240405', 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W98\\ica20240405\\ica01Data.csv')
('ICA', 'ica20240405', 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W98\\ica20240405\\ica02Data.csv')
('ICA', 'ica20240405', 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W98\\ica20240405\\ica03Data.csv')
('ICA', 'ica20240407', 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W98\\ica20240407\\ica01Data.csv')
('ICA', 'ica20240407', 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W98\\ica20240407\\ica02Data.csv')
('ICA', 'ica20240407', 'c:\\Users\

In [2]:
import os
import pandas as pd

root = 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99'
paths= []

for item in os.listdir(root):
    item_path = os.path.join(root, item)
    if os.path.isfile(item_path):
        paths.append(item_path)

paths

['C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_CK.csv',
 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_DL.csv',
 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_ICA.csv',
 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_TJ.csv',
 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_TPY.csv',
 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_TS.csv',
 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_WH.csv',
 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\processed\\W99\\Igv_W99_YH.csv']

In [ ]:
for i in range(1,len(paths)):
    df = pd.read_csv(paths[i], index_col= 0)
    # df['loc_len'] = df['target_location'].str.len()
    # col = ['loc_len', 'target_location']
    # # print(df.columns)
    print(paths[i])
    # print(df['target_location'].isna().value_counts())
    # print(df[col].value_counts(), '\n')

    print(df['mission_type'].unique().tolist())
    print(df['task_stage'].unique().tolist())
    print(df['current_task'].unique().tolist())

    print()


In [ ]:
display(df['mission_type'].unique())
display(df['task_stage'].unique())
display(df['current_task'].unique())

In [11]:
import os
icap = 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\westwell_da\\data\\raw\\W99\\ica20240416\\ica01Data.csv'
icap = os.path.abspath(icap)

def __strip_str__(df=pd.DataFrame):
    """
    Strip leading and trailing whitespaces from column names and string columns.

    Parameters:
    DataFrame (pd.DataFrame): Input DataFrame. 

    Returns:
    pd.DataFrame: DataFrame with leading and trailing whitespaces stripped from column names and string columns.
    """
    df.columns = df.columns.str.strip()
    
    # Select columns of type 'object'
    obj_cols = df.select_dtypes(['object']).columns
    
    # Strip leading and trailing whitespaces from string columns
    df[obj_cols] = df[obj_cols].apply(lambda x: x.str.strip())

    return df

df = pd.read_csv(icap, index_col=0)
df = __strip_str__(df)
print(df.columns, len(df))
df['local_time'] = pd.to_datetime(df['local_time'], format='%Y-%m-%d %H:%M:%S')

Index(['position_x', 'position_y', 'position_z', 'heading', 'speed',
       'speed_command', 'mileage_distance', 'soc', 'local_time',
       'chassis_mode', 'estop', 'charge_status', 'task_state_running',
       'task_state_estop', 'task_state_lock', 'task_stage', 'current_task',
       'error_code', 'target_location', 'vesselVisitID', 'mission_type',
       'container1_type', 'container2_type', 'vehicle_mode', 'missionID',
       'lane_change_state'],
      dtype='object') 5461


In [15]:
def ica_processer(df=pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values('local_time').reset_index(drop=True)
    df = get_location2(df=df)
    df['location_tag'] = df['target_location'].apply(lambda x: lmd_get_location_tag(x))
    df = get_current_task_tag(df=df)
    df['mission_type'] = df.apply(lambda x: lmd_get_mission_type(x['current_task_tag'], x['location_tag']), axis=1)
    df = get_cycle(df=df)
    return df


def get_location2(df=pd.DataFrame):
    df['target_location2'] = df['target_location'].str.split(r'\， ').str[1].fillna('')
    df['target_location'] = df['target_location'].str.split(r'\， ').str[0]
    return df

def lmd_get_location_tag(target_location=str) -> str:
        if "crane" in target_location:      return 'QCTP'
        elif "block" in target_location:    return 'YARD'
        elif 'ts' in target_location:       return 'TS'
        else: return 'None'

def get_current_task_tag(df=pd.DataFrame):
    df['current_task_tag'] = df['mission_type'].apply(lambda x: 'LOAD' if x=='VSLD' else 'DSCH')
    return df

def lmd_get_mission_type(current_task_tag=str, location_tag=str) :
    if current_task_tag=='LOAD':
        if location_tag=='YARD':    return 'RECEIVE'
        elif (location_tag=='QCTP') | (location_tag=='TS'):  return 'DELIVER'
    elif current_task_tag=='DSCH':
        if (location_tag=='YARD') | (location_tag=='TS'):    return 'DELIVER'
        elif location_tag=='QCTP':  return 'RECEIVE'
    else: return
    
def get_cycle(df=pd.DataFrame):
    df = df.sort_values(by='local_time').reset_index(drop=True)
    df['cycle'] = ''
    index_li, data_li = [], []
    
    for task_tag, data in df.groupby(by='current_task_tag'):
        start_ind = data[data['mission_type']=='RECEIVE'].first_valid_index()
        data = data[data.index>=start_ind]
        data['mission_type_change'] = data['mission_type'] != data['mission_type'].shift(1)

        rows = data[data['mission_type_change']==True].index.tolist()
        if len(rows) %2 != 0:
            last_ind = rows[-1]
            data = data[data.index < last_ind]
            rows.remove(last_ind)

        data_li.append(data)
        index_li.append(rows)
    
    index_li = [item for sublist in index_li for item in sublist]
    cycle_ids = [i+1 if i%2!=0 else i for i in range(1, len(index_li)+1)] 
    cycle_ids = [val//2 for val in cycle_ids]
    cycle_dict = {index_li[i]: cycle_ids[i] for i in range(len(index_li))}
    
    df = pd.concat(data_li, axis=0)
    df['cycle'] = df.index.map(cycle_dict)    
    df['cycle'] = df['cycle'].ffill()
    df = df.reset_index(drop=True)

    return df

In [16]:
ica_processer(df)

,position_x,position_y,position_z,heading,speed,speed_command,mileage_distance,soc,local_time,chassis_mode,...,container1_type,container2_type,vehicle_mode,missionID,lane_change_state,target_location2,location_tag,current_task_tag,cycle,mission_type_change
0,1053.120842,64.834501,0.052069,-1.366301,-0.000048,0.0,952.0,54,2024-04-16 00:18:30,1,...,40,0,2,1094651,0,,QCTP,DSCH,1.0,True
1,1053.122840,64.833605,0.050427,-1.366266,0.012764,0.0,952.0,54,2024-04-16 00:18:33,1,...,40,0,2,1094651,0,,QCTP,DSCH,1.0,False
2,1053.255595,64.229949,0.048784,-1.366109,0.710693,0.0,952.0,54,2024-04-16 00:18:36,1,...,40,0,2,1094651,0,,QCTP,DSCH,1.0,False
3,1054.142703,59.922018,0.056125,-1.367333,1.064792,0.0,952.0,54,2024-04-16 00:18:39,1,...,40,0,2,1094651,0,,QCTP,DSCH,1.0,False
4,1055.035682,55.393657,0.060267,-1.374487,1.614976,0.0,952.0,54,2024-04-16 00:18:42,1,...,40,0,2,1094651,0,,QCTP,DSCH,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,1123.352174,124.838545,0.043419,1.773351,0.001540,0.0,960.0,38,2024-04-16 04:22:54,1,...,40,0,2,1095000,0,,QCTP,LOAD,215.0,False
5105,1123.339927,124.845262,0.044661,1.773233,-0.000413,0.0,960.0,38,2024-04-16 04:22:57,1,...,40,0,2,1095000,0,,QCTP,LOAD,215.0,False
5106,1123.344327,124.854622,0.045238,1.773278,-0.000680,0.0,960.0,38,2024-04-16 04:23:00,1,...,40,0,2,1095000,0,,QCTP,LOAD,215.0,False
5107,1123.338393,124.858499,0.045549,1.773298,-0.000076,0.0,960.0,38,2024-04-16 04:23:09,1,...,40,0,2,1095009,0,,YARD,LOAD,215.0,True
